In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\0\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\0\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[84.0, 66.0, 100.0, 16.0, 95.0, 10.0, 13.0, 37.0, 13.0, 45.0, 55.0, 100.0, 32.0, 23.0, 100.0, 86.0, 4.0, 32.0, 47.0, 39.0, 100.0, 66.0, 7.0, 37.0, 16.0, 16.0, 100.0, 32.0, 45.0, 13.0, 84.0, 16.0, 86.0, 20.0, 10.0, 95.0, 37.0, 66.0, 100.0, 86.0, 47.0, 66.0, 96.0, 95.0, 86.0, 30.0, 45.0, 80.0, 66.0, 45.0, 95.0, 30.0, 96.0, 37.0, 37.0, 40.0, 40.0, 45.0, 4.0, 37.0, 100.0, 20.0, 86.0, 4.0, 20.0, 96.0, 86.0, 100.0, 7.0, 52.0, 100.0, 80.0, 100.0, 100.0, 10.0, 39.0, 37.0, 13.0, 37.0, 95.0, 32.0, 95.0, 10.0, 55.0, 23.0, 37.0, 40.0, 32.0, 66.0, 80.0, 55.0, 37.0, 52.0, 37.0, 96.0, 7.0, 37.0, 100.0, 93.0, 95.0, 10.0, 95.0, 16.0, 47.0, 47.0, 32.0, 100.0, 45.0, 40.0, 86.0, 95.0, 37.0, 45.0, 55.0, 30.0, 84.0, 52.0, 55.0, 13.0, 37.0, 95.0, 45.0, 10.0, 10.0, 52.0, 37.0, 52.0, 13.0, 40.0, 37.0, 10.0, 52.0, 84.0, 55.0, 10.0, 100.0, 100.0, 47.0, 55.0, 45.0, 93.0, 32.0, 86.0, 80.0, 13.0, 86.0, 47.0, 13.0, 45.0, 4.0, 47.0, 80.0, 93.0, 45.0, 66.0, 45.0, 55.0, 37.0, 84.0, 86.0, 95.0, 37.0, 52.0, 96.0, 55.0, 3

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetLarge = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetLarge.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetLarge(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

343610240/343610240 [==============================] - 43s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 4033      
                                                                 
Total params: 84,920,851
Trainable params: 84,724,183
Non-trainable params: 196,668
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetLarge_loss_0.txt',history.history['loss'])
#要改
np.savetxt('NASNetLarge_valloss_0.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Tue Apr 18 17:05:16 2023
Epoch 1/300
591/591 [==============================] - ETA: 0s - loss: 84.0275

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt\assets


591/591 [==============================] - 463s 705ms/step - loss: 84.0275 - val_loss: 9090.8203
Epoch 2/300
591/591 [==============================] - ETA: 0s - loss: 2.2217

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt\assets


...

591/591 [==============================] - 400s 678ms/step - loss: 0.0109 - val_loss: 0.4351
Epoch 24/300
591/591 [==============================] - 267s 452ms/step - loss: 0.0104 - val_loss: 1.1744
Epoch 25/300
591/591 [==============================] - 272s 460ms/step - loss: 0.0095 - val_loss: 0.7788
Epoch 26/300
591/591 [==============================] - 273s 463ms/step - loss: 0.0172 - val_loss: 0.7538
Epoch 27/300
591/591 [==============================] - ETA: 0s - loss: 0.0161

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\0\NASNetLarge.ckpt\assets


591/591 [==============================] - 409s 693ms/step - loss: 0.0161 - val_loss: 0.2442
Epoch 28/300
591/591 [==============================] - 282s 477ms/step - loss: 0.0071 - val_loss: 0.8130
Epoch 29/300
591/591 [==============================] - 277s 468ms/step - loss: 0.0060 - val_loss: 0.5015
Epoch 30/300
591/591 [==============================] - 282s 477ms/step - loss: 0.0067 - val_loss: 0.5416
Epoch 31/300
591/591 [==============================] - 505s 854ms/step - loss: 0.0052 - val_loss: 0.7028
Epoch 32/300
591/591 [==============================] - 487s 825ms/step - loss: 0.0137 - val_loss: 0.8861
Epoch 33/300
591/591 [==============================] - 337s 571ms/step - loss: 0.0082 - val_loss: 1.1629
Epoch 34/300
591/591 [==============================] - 261s 442ms/step - loss: 0.0169 - val_loss: 0.5398
Epoch 35/300
591/591 [==============================] - 260s 440ms/step - loss: 0.0056 - val_loss: 0.6466
Epoch 36/300
591/591 [==============================] - 261

591/591 [==============================] - 263s 444ms/step - loss: 0.0167 - val_loss: 0.4133
Epoch 179/300
591/591 [==============================] - 262s 444ms/step - loss: 6.1567e-04 - val_loss: 0.4616
Epoch 180/300
591/591 [==============================] - 263s 445ms/step - loss: 4.6456e-04 - val_loss: 0.4284
Epoch 181/300
591/591 [==============================] - 262s 444ms/step - loss: 4.8136e-04 - val_loss: 0.5639
Epoch 182/300
591/591 [==============================] - 263s 445ms/step - loss: 0.0013 - val_loss: 0.5273
Epoch 183/300
591/591 [==============================] - 263s 444ms/step - loss: 3.8333e-04 - val_loss: 0.5245
Epoch 184/300
591/591 [==============================] - 262s 443ms/step - loss: 4.7054e-04 - val_loss: 0.5584
Epoch 185/300
591/591 [==============================] - 262s 444ms/step - loss: 3.5155e-04 - val_loss: 0.5621
Epoch 186/300
591/591 [==============================] - 262s 444ms/step - loss: 0.0042 - val_loss: 0.4431
Epoch 187/300
591/591 [====